In [ ]:
# so I need folder structure like
# train/nyc/...

# test/nyc/...


# 0069 — nyc
# 0075 — las vegas
# 0084 — firenca
# 0093 — amsterdam

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
import subprocess
import tarfile
from math import ceil
import os
from tqdm import tqdm
import shutil
import torch
import torchvision
from torchvision import transforms

In [ ]:
cities = {'0075' : 'las vegas', '0083' : 'firenca', '0093' : 'amsterdam'}

In [ ]:
!mkdir train
!mkdir train/nyc
!mkdir train/las\ vegas
!mkdir train/firenca
!mkdir train/amsterdam

!mkdir test
!mkdir test/nyc
!mkdir test/las\ vegas
!mkdir test/firenca
!mkdir test/amsterdam

In [ ]:
tars = ['0075', '0083', '0093']
for tar_name in tqdm(tars):
  subprocess.call(f'gsutil cp gs://eccv16/dataset_unaligned/{tar_name}.tar . ', shell=True, executable='/bin/bash')

  tar = tarfile.open(f'{tar_name}.tar')
  tar.extractall('.') # specify which folder to extract to
  tar.close()

  subprocess.call(f'rm -rf {tar_name}.tar', shell=True, executable='/bin/bash')


100%|██████████| 3/3 [20:42<00:00, 414.18s/it]


In [ ]:
def distance(x, y):
  return (x[1] - y[1])**2 + (x[0] - y[0])**2

def coors(filename):
  with open(filename) as fp:
    f = fp.read()
    f = f.split(' ')
    f = f[5:7]
    f = [float(x) for x in f]
  return tuple(f)

def folders_to_coors(folder_name):
  coor_set = list()
  for file in os.listdir(folder_name):
    if file[-3:] == 'txt':
      try:
        coords = coors(f'{folder_name}/{file}')
      except UnicodeDecodeError:
        continue
      if coords not in coor_set:
        coor_set.append(coords)
  return coor_set

In [ ]:
for tar in tqdm(tars):
  coordinates = folders_to_coors(tar)

  train_set = []
  test_set = []
  chunks_size = ceil(0.1*len(coordinates))

  i = 0
  while coordinates != []:
    print(len(coordinates))
    anchor_point = min(coordinates, key=lambda x: x[0])
    distances = [distance(x, anchor_point) for x in coordinates]
    coors_and_dists = list(zip(coordinates, distances))
    coors_and_dists = sorted(coors_and_dists, key=lambda x: x[1])
    first_chunk = coors_and_dists[:chunks_size]
    second_chunk = coors_and_dists[chunks_size:]
    chunk = [x[0] for x in first_chunk]
    coordinates = [x[0] for x in second_chunk]
    if i % 2 == 0:
      train_set += chunk
    else:
      test_set += chunk
    i += 1

  split = {'train' : [], 'test' : []}
  for file in os.listdir(tar):
    if file[-3:] == 'txt':
      try:
        coords = coors(f'{tar}/{file}')
      except UnicodeDecodeError:
        continue
      if coords in train_set:
        split['train'].append(file[:-4])
      else:
        split['test'].append(file[:-4])

  split['train'] = [x + '.jpg' for x in split['train']]
  split['test'] = [x + '.jpg' for x in split['test']]

  for f in split['train']:
    try:
      shutil.move(os.path.join(tar, f), os.path.join('train', cities[tar]))
    except FileNotFoundError:
      continue  

  for f in split['test']:
    try:
      shutil.move(os.path.join(tar, f), os.path.join('test', cities[tar]))
    except FileNotFoundError:
      continue  

    


  0%|          | 0/3 [00:00<?, ?it/s]

3416
3074
2732
2390
2048
1706
1364
1022
680
338


 33%|███▎      | 1/3 [02:39<05:18, 159.48s/it]

5677
5109
4541
3973
3405
2837
2269
1701
1133
565


 67%|██████▋   | 2/3 [06:01<03:04, 184.24s/it]

6302
5671
5040
4409
3778
3147
2516
1885
1254
623


100%|██████████| 3/3 [07:34<00:00, 151.50s/it]


## ML part

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
transform = transforms.Compose([
    # you can add other transformations in this list
    transforms.ToTensor()
])

train_data = torchvision.datasets.ImageFolder(root='train', transform=transform)
test_data = torchvision.datasets.ImageFolder(root='test', transform=transform)

In [ ]:
batch_size = 64
train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(dataset=test_data, batch_size=batch_size, shuffle=True, num_workers=2)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc3 = nn.Linear(394384, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc3(x))
        return x


net = Net()
net = net.to(device)

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs, labels = inputs.cuda(), labels.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 50 == 49:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 50:.3f}')
            running_loss = 0.0

print('Finished Training')

[1,    50] loss: 1.361
[1,   100] loss: 0.938
[1,   150] loss: 0.861
[1,   200] loss: 0.842
[1,   250] loss: 0.834
[1,   300] loss: 0.779
[1,   350] loss: 0.805
[1,   400] loss: 0.774
[1,   450] loss: 0.749
[1,   500] loss: 0.746
[1,   550] loss: 0.703
[1,   600] loss: 0.698
[1,   650] loss: 0.674
[1,   700] loss: 0.687
[1,   750] loss: 0.659
[1,   800] loss: 0.707
[1,   850] loss: 0.628
[1,   900] loss: 0.629
[1,   950] loss: 0.637
[1,  1000] loss: 0.612
[1,  1050] loss: 0.590
[1,  1100] loss: 0.603
[1,  1150] loss: 0.580
[1,  1200] loss: 0.569
[1,  1250] loss: 0.556
[1,  1300] loss: 0.562
[1,  1350] loss: 0.551
[1,  1400] loss: 0.544
[1,  1450] loss: 0.565
[1,  1500] loss: 0.521
[1,  1550] loss: 0.534
[1,  1600] loss: 0.511
[1,  1650] loss: 0.531
[1,  1700] loss: 0.502
[1,  1750] loss: 0.525
[1,  1800] loss: 0.474
[1,  1850] loss: 0.516
[1,  1900] loss: 0.483
[1,  1950] loss: 0.486
[1,  2000] loss: 0.479
[1,  2050] loss: 0.471
[1,  2100] loss: 0.453
[1,  2150] loss: 0.483
[1,  2200] 

In [ ]:
PATH = './fourcities_net.pth'
torch.save(net.state_dict(), PATH)

In [ ]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images, labels = images.cuda(), labels.cuda()
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the test images: {100 * correct // total} %')

Accuracy of the network on the test images: 83 %
